In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# Cours TAL – Mini-projet
# Classification de dépêches d’agence avec NLTK

> Tiago Povoa Quinteiro

**Modalités du projet**

L’objectif de ce projet est de réaliser des expériences de classification de documents sous NLTK avec
le corpus de dépêches Reuters. Le projet est individuel : vous êtes responsable des différentes options
choisies, et en principe les résultats de chaque projet seront différents. Le projet sera jugé sur la
qualité des expériences (correction méthodologique) mais aussi sur la discussion des options
explorées.

Vous devez remettre un notebook Jupyter présentant vos choix, votre code, vos résultats et les
discussions. Le notebook devra déjà contenir les résultats des exécutions, mais pourra être ré-exécuté
par le professeur en vue d’une vérification.

Vous devrez en outre faire une courte présentation orale (5-7 min.) et répondre aux questions sur
votre projet (5-7 min.) lors d’une séance sur Teams (15 min.) avec le professeur et l’assistant.

**Description des expériences**

1. **L’objectif général** est d’explorer au moins deux aspects parmi les multiples choix qui se posent lors de la création d’un système de classification de textes.
2. **Données** : les dépêches du corpus Reuters, tel qu’il est fourni par NLTK. Vous respecterez notamment la division en données d’entraînement (train) et données de test.
3. **Hyper-paramètres** : la définition d’un classifieur comporte un grand nombre de choix de conception, dans plusieurs dimensions. Dans ce projet, et pour chaque objectif de classification (voir ci-dessous) vous explorerez deux dimensions. Pour chaque dimension, vous comparerez au moins deux options pour trouver laquelle fournit le meilleur score, et vous tenterez d’expliquer pourquoi. Vous pourrez choisir parmi les options suivantes :

    a. options de prétraitement des textes : stopwords, lemmatisation, tout en minuscules.
    
    b. options de représentation : présence/absence de mots indicateurs, nombre de mots indicateurs ; présence/absence/nombre de bigrammes, trigrammes ; autres traits : longueur de la dépêche, rapport tokens/types.
    
    c. classifieurs et leurs paramètres : divers choix possibles (voir la documentation).
    
    
4. **Objectif de classification** : vous devrez construire quatre classifieurs. Vous choisirez les meilleurs hyper-paramètres pour chaque classifieur sans regarder les résultats sur les données de test NLTK, mais en divisant les données d’entraînement NLTK en 80% train et 20% dev. Vous ferez ensuite l’entraînement final sur l’intégralité des données d’entraînement.

    a. Veuillez d’abord définir et entraîner trois classifieurs binaires, correspondant à trois catégories de votre choix. Chaque classifieur prédit si une dépêche appartient ou non à la catégorie, i.e. si elle doit recevoir ou non l’étiquette respective. Veuillez construire un premier classifieur binaire pour une étiquette que vous choisirez librement parmi les trois suivantes : ‘money-fx’, ‘interest’, ou ‘money-supply’. Le deuxième classifieur binaire concernera une étiquette de votre choix parmi : ‘grain’, ‘wheat’, ‘corn’. Enfin, le troisième sera choisi parmi : ‘crude’, ‘nat-gas’, ‘gold’.
        - Veuillez donner les scores de rappel, précision et f-mesure de chacun des trois classifieurs que vous avez conçus et entraînés.
    
    b. On vous demande également de définir un quatrième classifieur qui assigne l’une des trois étiquettes que vous avez choisies ci-dessus plus la catégorie ‘other’ (il assigne donc une seule étiquette parmi quatre). Vous devrez adapter légèrement les données, car un très petit nombre de dépêches (combien ?) sont en réalité annotées avec plusieurs de ces étiquettes, et vous n’en retiendrez que la première.
        - Veuillez évaluer ce classifieur en termes de rappel, précision et f-mesure pour chacune des trois étiquettes choisies ci-dessus, et comparer ces trois scores à ceux des trois classifieurs binaires précédents.
    
5. **Documentation** : livre NLTK, chapitre 2 pour le corpus Reuters, chapitre 6 pour la classification, et http://www.nltk.org/howto/classify.html pour les classifieurs dans NLTK ; Introduction to Information Retrieval (https://nlp.stanford.edu/IR-book/information-retrieval-book.html), chapitre 13, pour une discussion de méthodes de classification, et des exemples de scores obtenus sur certaines étiquettes.

## Function definitions

Here are some util functions to remove stop words, lemmatization, punctuation, ...

In [3]:
import nltk
from nltk.corpus import reuters
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import string

In [4]:
def remove_stop_words(words):
    """
    Remove english stopwords and words of length 1
    inspiration: http://www.nltk.org/book/ch02.html
    words: a list of words
    return words lower case without stopwords
    """
    stopwords = nltk.corpus.stopwords.words('english')
    return [w.lower() for w in words if len(w) > 1 and w.lower() not in stopwords]


def remove_punctuation(words):
    """
    inspiration: https://machinelearningmastery.com/clean-text-machine-learning-python/
    """
    table = str.maketrans('', '', string.punctuation)
    return [w.translate(table) for w in words]


def do_lemmatize(words):
    wnl = WordNetLemmatizer() 
    return [wnl.lemmatize(w) for w in words]


def filter_words(words, rm_punctuation, rm_stopwords, lemmatize):
    if rm_punctuation:
        words = remove_punctuation(words)
        
    if lemmatize:
        words = do_lemmatize(words)

    if rm_stopwords:
        words = remove_stop_words(words)
            
    return words


def get_reuters_vocab(sample_size, rm_punctuation, rm_stopwords, lemmatize):
    """
    return a sample of words from reuters
    """
    words = reuters.words()         
    
    fdist = FreqDist(words)
    
    print('Number of words in reuters: {} total - unique: {} \n'.format(fdist.N(), fdist.B()))
    
    fdist = FreqDist(filter_words(words, rm_punctuation, rm_stopwords, lemmatize) )

    print('After removals: {} total - unique: {} \n'.format(fdist.N(), fdist.B()))

    return [w[0] for w in fdist.most_common(sample_size)]


def vocab_dic(vocab):
    """
    return a vocabulary list as a dictionnary with all values
    set to False
    """
    return {w:False for w in vocab}


reuters_vocabulary_dic = vocab_dic(get_reuters_vocab(500, True, True, True))

Number of words in reuters: 1720901 total - unique: 41600 

After removals: 961132 total - unique: 29649 



In [5]:
def _category(fileid, wanted_category):
    """
    return a specific category given a fileid. If not found, returns "not_category"
    """
    return wanted_category if wanted_category in reuters.categories(fileid) else f'not_{wanted_category}'


def _words_dic(vocab_dic, words):
    _tmp_dic = dict()
    _tmp_dic.update(vocab_dic)
    
    for w in words:
        if w in vocab_dic:
            _tmp_dic[w] = True
            
    return _tmp_dic
            

def prepare_reuters_data(vocab_dic, wanted_category, rm_punctuation, rm_stopwords, lemmatize):
    __train_data = []
    __test_data = []

    for fileid in reuters.fileids():
        category = _category(fileid, wanted_category)
        words = reuters.words(fileid)
        words = filter_words(words, rm_punctuation, rm_stopwords, lemmatize)
        dic = _words_dic(vocab_dic, words)
        if fileid.startswith('test'):
            __train_data.append([dic, category])
        else:
            __test_data.append([dic, category])
                
    return __test_data, __test_data

In [6]:
def prepare_pre_cut(train_data, test_data):
    """
    As asked by the assignement, we have to cut on train data
    between 80% for training and 20% as dev (here named test)
    """
    cut_train = int(len(train_data) * 0.8 )
    # cut_test = int(len(train_data) * 0.2 )
    return train_data[0:cut_train], train_data[cut_train:]


reuters_train_data, reuters_test_data = prepare_reuters_data(reuters_vocabulary_dic, 'money-supply', True, True, True)
print(len(reuters_train_data), len(reuters_test_data))

cut_reuters_train_data, cut_reuters_test_data = prepare_pre_cut(reuters_train_data, reuters_test_data)
print(len(cut_reuters_train_data), len(cut_reuters_test_data))

7769 7769
6215 1554


In [7]:
from nltk.classify import * 

In [8]:
# classifier_2 = nltk.NaiveBayesClassifier.train(cut_reuters_train_data)
# print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier_2, cut_reuters_test_data))*100)
# classifier_2.show_most_informative_features(10)

In [9]:
import collections 
from nltk.metrics import precision, recall, f_measure

In [12]:
def obtain_data(wanted_category, cut, sampleSize, rm_punctuation, rm_stopwords, lemmatize):
    _vocab_dic = vocab_dic(get_reuters_vocab(sampleSize, rm_punctuation, rm_stopwords,  lemmatize))
    
    _train_data, _test_data = prepare_reuters_data(_vocab_dic, wanted_category, rm_punctuation, rm_stopwords,  lemmatize)

    if cut:
        _train_data, _test_data = prepare_pre_cut(_train_data, _test_data)
        
    return _train_data, _test_data
    
    
def eval_classifier(trained_classifier, test_data, label):
    """
    Inspiration: https://streamhacker.com/2010/05/17/text-classification-sentiment-analysis-precision-recall/
    """
    ref_set = collections.defaultdict(set)
    test_set = collections.defaultdict(set)
    
    for i, (feats, true_label) in enumerate(test_data):
        ref_set[true_label].add(i)
        observed = trained_classifier.classify(feats)
        
        test_set[observed].add(i)

    print('Classifier evaluation: ')
    print( 'Precision:', precision(ref_set[label], test_set[label]) )
    print( 'Recall:', recall(ref_set[label], test_set[label]) )
    print( 'F-measure:', f_measure(ref_set[label], test_set[label]) )
    
    
def run_classifier(classifier, wanted_category, cut=True, sampleSize=500, rm_punctuation=True, rm_stopwords=True, lemmatize=True):
    train_data, test_data = obtain_data(wanted_category, cut, sampleSize, rm_punctuation, rm_stopwords,  lemmatize)
    
    print(f'{classifier}')
    
    trained_classifier = None
    if f'{classifier}' == "<class 'nltk.classify.maxent.MaxentClassifier'>":
        print('Hello Maxent')
        trained_classifier = classifier.train(train_data, trace=3)
    else:
        trained_classifier = classifier.train(train_data)
    
    eval_classifier(trained_classifier, test_data, wanted_category)

    
categories = {
    0: ['money-fx', 'interest', 'money-supply'],
    1: ['grain', 'wheat', 'corn'],
    2: ['crude', 'nat-gas', 'gold']
}

# Part 4a

## Naive Bayes classifier

> Chosen category: money-fx


In [60]:
# Naive Bayes classifier
# Hyper paramters
CUT=True
SAMPLE_SIZE=10
RM_PONCTUATION=True
RM_STOPWORDS=True
LEMMATIZE=False

run_classifier(NaiveBayesClassifier, categories[0][0], CUT, SAMPLE_SIZE, RM_PONCTUATION, RM_STOPWORDS, LEMMATIZE)

Number of words in reuters: 1720901 total - unique: 41600 

After removals: 964625 total - unique: 30778 

<class 'nltk.classify.naivebayes.NaiveBayesClassifier'>
Classifier evaluation: 
Precision: 0.2757009345794392
Recall: 0.4306569343065693
F-measure: 0.3361823361823361


In the examples below, the data is always CUT=True
The category word chosen was 'money-fx' since it was the first. Why not?

### Test 1: Filtering

#### Variant 1: No filter

* SAMPLE_SIZE=5000

Every other parameter to false


* Precision: 0.3879003558718861
* Recall: 0.7956204379562044
* F-measure: 0.5215311004784688

#### Variant 2: Punctuation

Adding the punctuation filter

* Precision: 0.39636363636363636
* Recall: 0.7956204379562044
* F-measure: 0.529126213592233

Adding the punctuation filter improves a little bit the precision.

#### Variant 3: Stop word

* Precision: 0.4703196347031963
* Recall: 0.7518248175182481
* F-measure: 0.5786516853932584

It does significantly improve the precision.
The recall is a bit lower, the F-Measure improves (since we improve one of it's parameters)

#### Variant 4: Lemmatization

* Precision: 0.39636363636363636
* Recall: 0.7956204379562044
* F-measure: 0.529126213592233

This variant looks very similar to punctuation. We lose too much by not using stop words

#### Variant 5: Lemmatization and Stop word

* Precision: 0.46017699115044247
* Recall: 0.7591240875912408
* F-measure: 0.5730027548209367

By adding lemmatization, we lose a tiny bit on precision and a tinier bit in recall.
Since F-Measure is a metric using both, we shall use it to decide: It seems it does not improve.

#### Conclusion

The better F-Measure was variant 3 (punctuation, stop words, no lemmatization).

### Test 2:  Vocabulary size

#### 500 

* Precision: 0.45564516129032256
* Recall: 0.8248175182481752
* F-measure: 0.587012987012987

#### 1250

* Precision: 0.4826086956521739
* Recall: 0.8102189781021898
* F-measure: 0.6049046321525885

#### 2500

* Precision: 0.5045871559633027
* Recall: 0.8029197080291971
* F-measure: 0.6197183098591549

#### 5000

* Precision: 0.4703196347031963
* Recall: 0.7518248175182481
* F-measure: 0.5786516853932584

### Conclusion

The best score was obtained with 2500 in Vocabulary size and by filtering stop words.


## Decision Tree Classifier

> chosen category: corn

In [67]:
# Decision Tree Classifier
# Hyper paramters
CUT=True
SAMPLE_SIZE=1250
RM_PONCTUATION=True
RM_STOPWORDS=False
LEMMATIZE=True

run_classifier(DecisionTreeClassifier, categories[1][2], CUT, SAMPLE_SIZE, RM_PONCTUATION, RM_STOPWORDS, LEMMATIZE)

Number of words in reuters: 1720901 total - unique: 41600 

After removals: 1720901 total - unique: 39425 

<class 'nltk.classify.decisiontree.DecisionTreeClassifier'>
Classifier evaluation: 
Precision: 0.84
Recall: 0.6774193548387096
F-measure: 0.7499999999999999


### Test 1: Vocabulary size

#### Variant A: 5000 all filters

* Precision: 0.9333333333333333
* Recall: 0.9032258064516129
* F-measure: 0.9180327868852458

#### Variant B: 2500 all filters

* Precision: 0.9655172413793104
* Recall: 0.9032258064516129
* F-measure: 0.9333333333333333

#### Variant C: 1250 all filters

* Precision: 0.9666666666666667
* Recall: 0.9354838709677419
* F-measure: 0.9508196721311474

#### Variant D: 500 All filters

* Precision: 1.0
* Recall: 0.6774193548387096
* F-measure: 0.8076923076923077

### Test 2: 

* SAMPLE_SIZE=1250

#### Variant A: No filters

* Precision: 0.84
* Recall: 0.6774193548387096
* F-measure: 0.7499999999999999

#### Variant B: Punctuation

* Precision: 0.84
* Recall: 0.6774193548387096
* F-measure: 0.7499999999999999

Removing punctuation didn't change anything.

#### Variant C: Stop words

* Precision: 0.9666666666666667
* Recall: 0.9354838709677419
* F-measure: 0.9508196721311474

It is totally identical to the variant with all filters.

#### Variant D: Lemmatization (no stop words filtering)

* Precision: 0.84
* Recall: 0.6774193548387096
* F-measure: 0.7499999999999999

Adding lemmatization didn't help.

#### Conclusion

The only parameter who does help it the decision tree is removing stop words.
Adding the two other filters doesn't seem to change the scores.

### Conclusion

Best parameters: 1250 vocabulary size, Stop words on (you can add the other filters but it doesn't seem to change something)

## MaxentClassifier
> chose category: gold

In [13]:
# MaxentClassifier
# Hyper paramters
CUT=True
SAMPLE_SIZE=500
RM_PONCTUATION=True
RM_STOPWORDS=True
LEMMATIZE=True

run_classifier(MaxentClassifier, categories[2][2], CUT, SAMPLE_SIZE, RM_PONCTUATION, RM_STOPWORDS, LEMMATIZE)

Number of words in reuters: 1720901 total - unique: 41600 

After removals: 961132 total - unique: 29649 

<class 'nltk.classify.maxent.MaxentClassifier'>
Hello Maxent
  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.988
             2          -0.11210        0.988
             3          -0.03275        0.988
             4          -0.01793        0.988
             5          -0.01410        0.988
             6          -0.01287        0.988
             7          -0.01240        0.988
             8          -0.01221        0.988
             9          -0.01212        0.988
            10          -0.01208        0.988
            11          -0.01205        0.988
            12          -0.01204        0.988
            13          -0.01204        0.988
            14          -0.01203        0.988
            15          -0.01203        0.988
            16   

# Part 4b